<a href="https://colab.research.google.com/github/AlecTraas/computational-geo-lab/blob/main/Colab/Kai/quickhull_algo_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

In [2]:
def main(num_points):
  points = sorted([[rnd.random()*10, rnd.random()*10, rnd.random()*10] for _ in range(num_points)], key=lambda x: x[0])
  np_points = np.array(points)
  fig = plt.figure()
  ax = fig.add_subplot(projection='3d')
  ax.scatter(np_points[:,0], np_points[:,1], np_points[:,2])
  hull = ConvexHull(np_points)
  hull.quick_hull()
  np_hull_pts = np.array(hull.hull_points)
  np_hull_fcts = np.array(hull.hull_facets)
  ax.plot_trisurf(np_hull_pts[:, 0], np_hull_pts[:, 1], np_hull_pts[:, 2], triangles=np_hull_fcts, color='red', alpha=0.5)
  plt.show()

In [5]:
# I miss structs :(
class ConvexHull():
  def __init__(self, points):
    # [[x,y,z]...]
    self.points = points
    # [[x,y,z]...]
    self.hull_points = None
    # references indecies in the hull_points array
    # [[a,b,c]...]
    self.hull_facets = None
    # references indecies in the hull_points array
    # [[a,b]...]
    self.hull_ridges = None
    # references indecies in the hull_ridges array
    # [a,b,c...]
    self.neighbors = None

    self.above_tri_sets = None

  def quick_hull(self):
    # I assume, for the sake of simplicity, that there are no degeneracies (low-dimensional sets, coplanar points, fewer than four points in set)
    self.define_tetrahedron()
    self.form_above_tri_sets()
    self.clear_internal_points()
    while self.contains_valid_external_sets() > 0:
      for i,ats in enumerate(self.above_tri_sets):
        if len(ats) > 0:
          far_p = self.above_tri_sets[self.furthest_point(i)]
          horizon = self.calculate_horizon(far_p, i)
          self.construct_cone(far_p, horizon)
          self.form_above_tri_sets()

  def define_tetrahedron(self):
    temp = np.array(self.points)
    # ↓ Inefficient, but it works
    i = []
    k = temp[:,0].argmin()
    i.append(k)
    temp[k] = np.inf
    k = temp[:,1].argmin()
    i.append(k)
    temp[k] = np.inf
    k = temp[:,2].argmin()
    i.append(k)
    temp[temp == np.inf] = -np.inf
    temp[k] = -np.inf
    k = temp[:,1].argmax()
    i.append(k)
    i = sorted(i, reverse=True)
    self.hull_points = [self.points[j, :] for j in i]
    for j in i:
      self.points = np.delete(self.points,j,0)
    self.hull_points.sort(key=lambda x: x[0])
    self.hull_points = self.hull_points[::-1]
    self.hull_facets = [[0,2,1],[0,1,3],[0,3,2],[1,2,3]]
    self.neighbors = [[[0,3,2],[1,2,3],[0,1,3]],
                      [[0,2,1],[1,2,3],[0,3,2]],
                      [[0,1,3],[1,2,3],[0,2,1]],
                      [[0,2,1],[0,3,2],[0,1,3]]]

  def above_facet(self,face,point):
    face = [self.hull_points[face[0]],self.hull_points[face[1]],self.hull_points[face[2]]]
    n = np.cross((face[1]-face[0]),(face[2]-face[0]))
    n = n/np.linalg.norm(n)
    d = np.dot(n,(face[0]-point))
    return d > 0

  def clear_internal_points(self):
      hull_centroid = np.sum(self.hull_points, axis=0) / len(self.hull_points)
      queue = []
      for p in self.points:
        for fc in self.facet_centroids():
          if np.dot(fc - hull_centroid, p - fc) > 0:
            queue.append(p)
      seen = set()
      queue = [x for x in queue if tuple(x) not in seen and not seen.add(tuple(x))]
      self.points = queue

  def face_to_point_list(self,f):
    p = self.hull_points
    return [p[i] for i in f]

  def facet_centroids(self):
    c = []
    for f in self.hull_facets:
      centroid = np.sum(np.array(self.face_to_point_list(f)), axis=0) / 3
      c.append(centroid.tolist())
    return c

  def furthest_point(self,face_index):
    furthest = 0
    for i,p in enumerate(self.above_tri_sets[face_index]):
      if self.point_face_dist(p,face_index) > furthest: # no idea how I'm supposed to get the perpendicular distance
        furthest = i
    return furthest

  def point_face_dist(self,point,face_index):
    pass

  def nearest_facet(self,point):
    min = np.inf
    out_i = 0
    for i,fc in enumerate(self.facet_centroids()):
      d = np.linalg.norm(point-fc)
      if d < min:
        min = d
        out_i = i
    return out_i

  def form_above_tri_sets(self):
    self.above_tri_sets = None
    for p in self.points:
      self.above_tri_sets[self.nearest_facet(p)].append(p)

In [ ]:
main(1000)